In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [1]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

NameError: name 'words' is not defined

In [5]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [],[]
for w in words:

    # print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
X.shape, X.dtype,Y.shape,Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
C = torch.randn((27, 2))

In [8]:
C[5]

tensor([2.2020, 0.4776])

In [9]:
# embedding: 就像是C是一个neural, 将x编码为ont hot 然后输入到neural中训练embedding
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([2.2020, 0.4776])

In [10]:
emb = C[X]
# 这里emb的单个shape为(3, 2), 三个字符，每个字符有一个x label, y, label
emb.shape

torch.Size([228146, 3, 2])

In [11]:
# cat会创建新的tensor,相比view比较消耗内存
# torch.cat(torch.unbind(emb, 1), 1).shape
# emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

In [12]:
# 因此这里的输入为6
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [13]:
# 这里需要对emb进行concat, 转为(32, 6)的矩阵
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [14]:
h.shape

torch.Size([228146, 100])

In [2]:
h

NameError: name 'h' is not defined

In [16]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [17]:
logits = h @ W2 + b2

In [18]:
logits.shape

torch.Size([228146, 27])

In [ ]:
# ------- final version ----------

In [22]:
# build the dataset 

def build_dataset(words):
    block_size = 3 # context length: how many characters do we take to predict the next one?
    X, Y = [],[]
    for w in words:

        # print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


import random
random.seed(42)
# 将words打乱
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [23]:
X.shape, Y.shape # dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [24]:
# 因此这里的输入为6
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [25]:
sum([p.nelement() for p in parameters])

11897

In [26]:
for p in parameters:
    p.requires_grad = True

In [27]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [28]:
lri = []
lossi = []
stepi = []

In [29]:
for i in range(200000):
    # mini batch
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # forward pass
    emb = C[Xtr[ix]] #(32, 3, 2)
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1) #(32, 100)
    logits = h @ W2 + b2 # (32, 27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Ytr[ix])
    # print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()


    # update 
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
    # lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())


# print(loss.item())

In [3]:
plt.plot(stepi, lossi)

NameError: name 'plt' is not defined

In [31]:
emb = C[Xtr] #(32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) #(32, 100)
logits = h @ W2 + b2 # (32, 27)
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdims=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1235, grad_fn=<NllLossBackward0>)

In [32]:
emb = C[Xdev] #(32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) #(32, 100)
logits = h @ W2 + b2 # (32, 27)
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdims=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1688, grad_fn=<NllLossBackward0>)

In [4]:
# visual the embeddings
plt.figure(figsize=(8,8))
plt.scatter(C[:, 0].data, C[:, 1].data, s = 200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha='center',va='center', color='white')
plt.grid('minor')

NameError: name 'plt' is not defined

In [ ]:
# train dataset, dev/validate dataset, test dataset
# 80%, 10%, 10%
# 测试集只能运行很少次，否则会出现从测试集学习的情况

In [42]:
# sample from models
g = torch.Generator().manual_seed(2147483647 + 12)
for i in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])] #(1, block_size,d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1) 
        logits = h @ W2 + b2 # (32, 27)
        probs = F.softmax(logits, dim=1)
        
        ix = torch.multinomial(probs, num_samples=1,generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
        
    print(''.join(itos[i] for i in out))

zaer.
ridgan.
sudo.
sci.
cereene.
garsha.
keyanith.
ulep.
prender.
sta.
urdan.
braylebelian.
pai.
avin.
zakerysley.
jone.
sera.
amanaviolameelah.
arha.
ezrimberk.


In [412]:
# C[torch.tensor([context])].view(-1, 30) == C[torch.tensor([context])].view(1, -1)

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True]])

In [38]:
emb.shape

torch.Size([1, 3, 10])